In [ ]:
"""Market_Cap_Sector_Industry_Classification.ipynb

This notebook enhances our insider trading dataset with company information
including market cap, sector, and industry classification.
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import os
from tqdm.notebook import tqdm  # Progress bar for loops
import pandas as pd
from io import BytesIO

In [ ]:
#If using conda update using the following commands:
#%conda install -c conda-forge ipywidgets
#%conda update jupyter

#If using pip, update using the following commands:
#%pip install ipywidgets --upgrade
#%pip install jupyter --upgrade

In [ ]:
# Load the insider trading dataset
print("Loading insider trading dataset...")
insider_df = pd.read_csv('notebook3_added_price_data.csv')

print("\nFirst 3 rows of the data:")
print(insider_df.head(3))

In [ ]:

# Install and import yfinance (only if needed)
try:
    import yfinance as yf
except ImportError:
    !pip install yfinance --upgrade --no-cache-dir
    import yfinance as yf

In [ ]:
def get_company_info(ticker):
    """
    Retrieves company information from Yahoo Finance

    Args:
        ticker (str): Stock ticker symbol

    Returns:
        dict: Dictionary with company information or None if retrieval fails
    """
    try:
        # Handle problematic tickers
        clean_ticker = ticker.strip().replace(' ', '')
        if not clean_ticker or clean_ticker.upper() in ['NONE', 'N/A']:
            return None

        # Some tickers may have special characters or formats that cause issues
        if '/' in clean_ticker:
            clean_ticker = clean_ticker.replace('/', '-')

        # Create ticker object
        ticker_obj = yf.Ticker(clean_ticker)

        # Get company info
        info = ticker_obj.info

        # Extract relevant fields
        company_data = {
            'sector': info.get('sector', None),
            'industry': info.get('industry', None),
            'market_cap': info.get('marketCap', None),
            'beta': info.get('beta', None),
            'pe_ratio': info.get('trailingPE', None),
            'forward_pe': info.get('forwardPE', None),
            'dividend_yield': info.get('dividendYield', None),
            'fifty_two_week_high': info.get('fiftyTwoWeekHigh', None),
            'fifty_two_week_low': info.get('fiftyTwoWeekLow', None),
            'company_name': info.get('longName', None)
        }

        return company_data

    except Exception as e:
        if "429" in str(e):
            print(f"Rate limit exceeded for {ticker}. Waiting...")
            time.sleep(2)  # Wait longer on rate limit errors
            return None
        else:
            # For other errors, just log and continue
            print(f"Error retrieving data for {ticker}: {e}")
            return None

# Test the function with a sample ticker that we know exists in the dataset
sample_ticker = insider_df['Ticker'].iloc[0]
print(f"\nTesting company info retrieval with ticker: {sample_ticker}")
sample_info = get_company_info(sample_ticker)
if sample_info:
    for key, value in sample_info.items():
        print(f"{key}: {value}")
else:
    print(f"Could not retrieve info for {sample_ticker}, trying another ticker...")
    sample_ticker = 'AAPL'  # Use a well-known ticker as backup
    sample_info = get_company_info(sample_ticker)
    if sample_info:
        print(f"Retrieved info for {sample_ticker} instead:")
        for key, value in sample_info.items():
            print(f"{key}: {value}")

In [ ]:
company_info_columns = [
    'sector', 'industry', 'market_cap', 'beta', 'pe_ratio',
    'forward_pe', 'dividend_yield', 'fifty_two_week_high',
    'fifty_two_week_low', 'company_name'
]

# Get unique tickers to minimize API calls
unique_tickers = insider_df['Ticker'].unique()
print(f"\nRetrieving company information for {len(unique_tickers)} unique tickers...")

# Create a dictionary to store company info by ticker
company_info_dict = {}

# Track successful retrievals
success_count = 0
total_count = 0

# Process all unique tickers with progress bar and rate limiting
for ticker in tqdm(unique_tickers):
    total_count += 1

    # Skip if we already have info for this ticker (sometimes duplicates exist)
    if ticker in company_info_dict:
        continue

    # Get company info
    company_info = get_company_info(ticker)

    # Store in dictionary if successful
    if company_info and any(company_info.values()):
        company_info_dict[ticker] = company_info
        success_count += 1


    # Print progress update every 50 tickers
    if total_count % 50 == 0:
        print(f"Progress: {success_count}/{total_count} successful ({success_count/total_count*100:.1f}%)")

print(f"Successfully retrieved info for {success_count} of {len(unique_tickers)} tickers ({success_count/len(unique_tickers)*100:.1f}%)")


In [ ]:
print("\nAdding company information to dataset...")

# Function to get company info from dictionary
def get_info_value(ticker, field):
    info = company_info_dict.get(ticker, {})
    return info.get(field, None)

# Create a copy of the dataframe to add our new columns
enhanced_df = insider_df.copy()

# Add each company info field to the dataset
for col in company_info_columns:
    enhanced_df[col] = enhanced_df['Ticker'].apply(lambda x: get_info_value(x, col))

# Display sample of enhanced dataset
print("\nSample of enriched dataset:")
print(enhanced_df.head(3))

# Display summary of data completeness
print("\nCompleteness of company information:")
for col in company_info_columns:
    not_null = enhanced_df[col].notna().sum()
    pct_complete = not_null / len(enhanced_df) * 100
    print(f"{col}: {not_null} entries ({pct_complete:.1f}%)")


In [ ]:
#---------- SECTION 6: MARKET CAP CATEGORIZATION ----------#
# Define market cap categories
def market_cap_category(cap):
    if pd.isna(cap) or cap is None:
        return "Unknown"
    elif cap < 2_000_000_000:
        return "Small cap ($0 - $2B)"
    elif cap < 10_000_000_000:
        return "Mid Cap ($2B - $10B)"
    else:
        return "Large Cap (> $10B)"

# Add market cap category
enhanced_df['market_cap_category'] = enhanced_df['market_cap'].apply(market_cap_category)

# Display distribution of market cap categories
print("\nDistribution of companies by market cap category:")
mc_counts = enhanced_df['market_cap_category'].value_counts()
print(mc_counts)

# Create plot of market cap categories
plt.figure(figsize=(10, 6))
mc_counts.plot(kind='bar', color='darkblue')
plt.title('Distribution of Insider Transactions by Market Cap Category')
plt.xlabel('Market Cap Category')
plt.ylabel('Number of Transactions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
#---------- SECTION 7: SECTOR ANALYSIS ----------#
# Display distribution of sectors
print("\nDistribution of companies by sector:")
sector_counts = enhanced_df['sector'].value_counts().head(10)
print(sector_counts)

# Create plot of top sectors
plt.figure(figsize=(12, 6))
sector_counts.plot(kind='bar', color='darkgreen')
plt.title('Top 10 Sectors with Insider Transactions')
plt.xlabel('Sector')
plt.ylabel('Number of Transactions')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Distribution of insider transactions by sector and market cap
print("\nNumber of transactions by sector and market cap category:")
sector_mc_counts = pd.crosstab(enhanced_df['sector'], enhanced_df['market_cap_category'])
print(sector_mc_counts)

# Plot heatmap of sector vs market cap
plt.figure(figsize=(12, 8))
sns.heatmap(sector_mc_counts, annot=True, cmap='YlGnBu', fmt='g')
plt.title('Distribution of Insider Transactions by Sector and Market Cap')
plt.tight_layout()
plt.show()

In [ ]:
#---------- SECTION 8: ADDITIONAL ANALYSIS WITH ENHANCED DATA ----------#
# Set styling for plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("deep")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Apply filters to avoid microstructure noise
# 1. Exclude stocks where transaction price per share < $5
# 2. Exclude stocks where market cap < $100 million
df_filtered = enhanced_df.copy()
if 'price_per_share' in df_filtered.columns:
    df_filtered = df_filtered[df_filtered['price_per_share'] >= 5]
if 'market_cap' in df_filtered.columns:
    df_filtered = df_filtered[df_filtered['market_cap'] >= 100_000_000]

print(f"After filtering: {df_filtered.shape}")
if df_filtered.shape[0] < enhanced_df.shape[0]:
    print(f"Removed {enhanced_df.shape[0] - df_filtered.shape[0]} rows ({(enhanced_df.shape[0] - df_filtered.shape[0])/enhanced_df.shape[0]*100:.1f}% of data)")

# Calculate additional metrics if price data is available
if 'return_6month_pct' in df_filtered.columns and 'spy_return_6month_pct' in df_filtered.columns:
    df_filtered['market_adjusted_return'] = df_filtered['return_6month_pct'] - df_filtered['spy_return_6month_pct']

# Add transaction year for time-based analysis if Transaction Date exists
if 'Transaction Date' in df_filtered.columns:
    df_filtered['Transaction_Year'] = pd.to_datetime(df_filtered['Transaction Date'],format="%d-%b-%Y").dt.year

In [ ]:
#---------- SECTION 9: SECTOR PERFORMANCE ANALYSIS ----------#
if 'return_6month_pct' in df_filtered.columns:
    # Filter out sectors with too few transactions
    min_transactions = 50
    sector_counts_analysis = df_filtered['sector'].value_counts()
    valid_sectors = sector_counts_analysis[sector_counts_analysis >= min_transactions].index

    if len(valid_sectors) > 0:
        # Calculate sector returns
        sector_returns = df_filtered[df_filtered['sector'].isin(valid_sectors)].groupby('sector').agg({
            'return_6month_pct': ['mean', 'median', 'std', 'count'],
            'Ticker': lambda x: x.nunique()  # Number of unique companies
        }).reset_index()

        # Flatten the multi-level columns
        sector_returns.columns = ['sector', 'mean_return', 'median_return', 'std_return', 'count', 'unique_companies']

        # Add success rate (% of positive returns)
        success_rates = df_filtered[df_filtered['sector'].isin(valid_sectors)].groupby('sector').apply(
            lambda x: (x['return_6month_pct'] > 0).mean() * 100).reset_index()
        success_rates.columns = ['sector', 'success_rate']

        sector_returns = sector_returns.merge(success_rates, on='sector')
        sector_returns = sector_returns.sort_values('mean_return', ascending=False)

        print("\n=== SECTOR PERFORMANCE ANALYSIS ===")
        print(sector_returns[['sector', 'mean_return', 'median_return', 'success_rate', 'count', 'unique_companies']])

        # Create bar chart of sector returns
        plt.figure(figsize=(14, 6))
        plt.bar(sector_returns['sector'], sector_returns['mean_return'], color='darkblue')
        plt.axhline(y=0, color='red', linestyle='--')
        plt.axhline(y=df_filtered['return_6month_pct'].mean(), color='green', linestyle='-',
                    label=f'Overall Mean: {df_filtered["return_6month_pct"].mean():.2f}%')
        plt.title('Average 6-Month Returns by Sector')
        plt.xlabel('Sector')
        plt.ylabel('Average Return (%)')
        plt.xticks(rotation=45, ha='right')
        plt.legend()
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.show()

In [ ]:
#---------- SECTION 10: INSIDER TITLE ANALYSIS ----------#
# Count unique values in Insider Title if it exists
if 'Insider Title' in enhanced_df.columns:
    title_counts = enhanced_df['Insider Title'].value_counts().reset_index()
    title_counts.columns = ['Title', 'Count']

    # Show the top 20 most common titles
    print("\nTop 20 most common Insider Titles:")
    print(title_counts.head(20))

In [ ]:
#---------- SECTION 11: SAVE ENHANCED DATASET ----------#
# Save the enhanced dataset
print("\nSaving enhanced dataset...")
try:
    # create a local copy for download
    enhanced_df.to_csv('notebook4_enhanced_corporate_info.csv', index=False)

    print("Enhanced dataset ready for download")

except Exception as e:
    print(f"Error saving file: {e}")

# Final summary
print(f"\n=== PROJECT SUMMARY ===")
print(f"- Started with {insider_df.shape[0]} transactions from Notebook 3")
print(f"- Retrieved company information for {success_count} of {len(unique_tickers)} unique companies")
print(f"- Added {len(company_info_columns)} new company information columns")
print(f"- Enhanced dataset shape: {enhanced_df.shape}")
print(f"- Output file: 'notebook4_enhanced_corporate_info.csv'")
print(f"- Ready for use in subsequent analysis notebooks")

In [ ]:
# Apply filters to avoid microstructure noise
# 1. Exclude stocks where transaction price per share < $5
# 2. Exclude stocks where market cap < $100 million (convert to same units)
df_filtered = enhanced_df.copy()
df_filtered = df_filtered[df_filtered['price_per_share'] >= 5]
df_filtered = df_filtered[df_filtered['market_cap'] >= 100_000_000]  # Assuming market cap is in dollars

print(f"After filtering: {df_filtered.shape}")
print(f"Removed {enhanced_df.shape[0] - df_filtered.shape[0]} rows ({(enhanced_df.shape[0] - df_filtered.shape[0])/enhanced_df.shape[0]*100:.1f}% of data)")

#------------------------#
# Calculate Key Metrics
#------------------------#
# Calculate market-adjusted returns
df_filtered['return_6month_pct'] = ((df_filtered['price_6month'] - df_filtered['average_price_per_share']) / 
                                    df_filtered['average_price_per_share']*100)
df_filtered['spy_return_6month_pct'] = ((df_filtered['spy_price_6month'] - df_filtered['spy_price_transactiondate']) /
                                        df_filtered['spy_price_transactiondate']*100)
df_filtered['market_adjusted_return'] = df_filtered['return_6month_pct'] - df_filtered['spy_return_6month_pct']

# Add transaction year for time-based analysis
df_filtered['Transaction_Year'] = pd.to_datetime(df_filtered['Transaction Date']).dt.year

#------------------------#
# 1. Overview of Data
#------------------------#
print("\n=== OVERVIEW OF FILTERED DATA ===")

# Summary statistics of returns
returns_summary = df_filtered[['return_6month_pct', 'spy_return_6month_pct', 'market_adjusted_return']].describe()
print("\nSummary statistics of returns (%):")
print(returns_summary)

# Distribution of transaction years
year_counts = df_filtered['Transaction_Year'].value_counts().sort_index()
plt.figure(figsize=(14, 6))
year_counts.plot(kind='bar', color='navy')
plt.title('Number of Insider Transactions by Year (Filtered Data)')
plt.xlabel('Year')
plt.ylabel('Number of Transactions')
plt.tight_layout()
plt.savefig('transactions_by_year.png', dpi=300)
plt.show()

#------------------------#
# 2. Return Distributions
#------------------------#
plt.figure(figsize=(14, 8))

# Top: Raw 6-month returns
plt.subplot(2, 1, 1)
sns.histplot(df_filtered['return_6month_pct'], bins=50, kde=True)
plt.axvline(x=0, color='red', linestyle='--')
plt.axvline(x=df_filtered['return_6month_pct'].mean(), color='green', linestyle='-',
            label=f'Mean: {df_filtered["return_6month_pct"].mean():.2f}%')
plt.title('Distribution of 6-Month Returns Following Insider Purchases')
plt.xlabel('Return (%)')
plt.xlim(-100, 100)  # Reasonable limits for readability
plt.legend()

# Bottom: Market-adjusted returns
plt.subplot(2, 1, 2)
sns.histplot(df_filtered['market_adjusted_return'], bins=50, kde=True)
plt.axvline(x=0, color='red', linestyle='--')
plt.axvline(x=df_filtered['market_adjusted_return'].mean(), color='green', linestyle='-',
            label=f'Mean: {df_filtered["market_adjusted_return"].mean():.2f}%')
plt.title('Distribution of Market-Adjusted 6-Month Returns')
plt.xlabel('Return - S&P 500 Return (%)')
plt.xlim(-100, 100)  # Reasonable limits for readability
plt.legend()

plt.tight_layout()
plt.savefig('return_distributions.png', dpi=300)
plt.show()

#------------------------#
# 3. Sector Analysis
#------------------------#
# Filter out sectors with too few transactions
min_transactions = 50
sector_counts = df_filtered['sector'].value_counts()
valid_sectors = sector_counts[sector_counts >= min_transactions].index

# Calculate sector returns - raw and market-adjusted
sector_returns = df_filtered[df_filtered['sector'].isin(valid_sectors)].groupby('sector').agg({
    'return_6month_pct': ['mean', 'median', 'std', 'count'],
    'market_adjusted_return': ['mean', 'median', 'std'],
    'Ticker': lambda x: x.nunique()  # Number of unique companies
}).reset_index()

# Flatten the multi-level columns
sector_returns.columns = ['sector', 'mean_return', 'median_return', 'std_return', 'count',
                         'mean_adj_return', 'median_adj_return', 'std_adj_return', 'unique_companies']

# Sort by mean market-adjusted return
sector_returns = sector_returns.sort_values('mean_adj_return', ascending=False)

# Add success rate (% of positive returns)
success_rates = df_filtered[df_filtered['sector'].isin(valid_sectors)].groupby('sector').apply(
    lambda x: (x['return_6month_pct'] > 0).mean() * 100).reset_index()
success_rates.columns = ['sector', 'success_rate']

sector_returns = sector_returns.merge(success_rates, on='sector')

print("\n=== SECTOR PERFORMANCE (FILTERED DATA) ===")
print(sector_returns[['sector', 'mean_return', 'median_return', 'success_rate',
                      'mean_adj_return', 'count', 'unique_companies']])

# Create bar chart of sector returns
plt.figure(figsize=(14, 10))

# Top plot: Raw returns
plt.subplot(2, 1, 1)
plt.bar(sector_returns['sector'], sector_returns['mean_return'], color='darkblue')
plt.axhline(y=0, color='red', linestyle='--')
plt.axhline(y=df_filtered['return_6month_pct'].mean(), color='green', linestyle='-',
            label=f'Overall Mean: {df_filtered["return_6month_pct"].mean():.2f}%')
plt.title('Average 6-Month Returns by Sector')
plt.xlabel('Sector')
plt.ylabel('Average Return (%)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', alpha=0.3)

# Bottom plot: Market-adjusted returns
plt.subplot(2, 1, 2)
plt.bar(sector_returns['sector'], sector_returns['mean_adj_return'], color='darkgreen')
plt.axhline(y=0, color='red', linestyle='--', label='S&P 500 Return')
plt.axhline(y=df_filtered['market_adjusted_return'].mean(), color='blue', linestyle='-',
            label=f'Overall Mean: {df_filtered["market_adjusted_return"].mean():.2f}%')
plt.title('Average Market-Adjusted Returns by Sector')
plt.xlabel('Sector')
plt.ylabel('Average Return - S&P 500 Return (%)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('sector_returns.png', dpi=300)
plt.show()

#------------------------#
# 4. Insider Role Analysis
#------------------------#
# Simplify the insider roles for better aggregation
df_filtered['Role_Simplified'] = df_filtered['Insider Role'].str.upper()
df_filtered.loc[df_filtered['Role_Simplified'].str.contains('CEO|CHIEF EXECUTIVE', na=False), 'Role_Simplified'] = 'CEO'
df_filtered.loc[df_filtered['Role_Simplified'].str.contains('CFO|CHIEF FINANCIAL', na=False), 'Role_Simplified'] = 'CFO'
df_filtered.loc[df_filtered['Role_Simplified'].str.contains('DIRECTOR', na=False), 'Role_Simplified'] = 'DIRECTOR'
df_filtered.loc[df_filtered['Role_Simplified'].str.contains('COO|CHIEF OPERATING', na=False), 'Role_Simplified'] = 'COO'
df_filtered.loc[df_filtered['Role_Simplified'].str.contains('PRESIDENT', na=False), 'Role_Simplified'] = 'PRESIDENT'
df_filtered.loc[df_filtered['Role_Simplified'].str.contains('VP|VICE PRESIDENT', na=False), 'Role_Simplified'] = 'VP'
df_filtered.loc[~df_filtered['Role_Simplified'].isin(['CEO', 'CFO', 'DIRECTOR', 'COO', 'PRESIDENT', 'VP']), 'Role_Simplified'] = 'OTHER'

# Calculate returns by insider role
role_returns = df_filtered.groupby('Role_Simplified').agg({
    'return_6month_pct': ['mean', 'median', 'std', 'count'],
    'market_adjusted_return': ['mean', 'median', 'std']
}).reset_index()

# Flatten multi-level columns
role_returns.columns = ['role', 'mean_return', 'median_return', 'std_return', 'count',
                        'mean_adj_return', 'median_adj_return', 'std_adj_return']

# Calculate success rates
role_success = df_filtered.groupby('Role_Simplified').apply(
    lambda x: (x['return_6month_pct'] > 0).mean() * 100).reset_index()
role_success.columns = ['role', 'success_rate']

role_returns = role_returns.merge(role_success, on='role')
role_returns = role_returns.sort_values('mean_adj_return', ascending=False)

print("\n=== INSIDER ROLE PERFORMANCE ===")
print(role_returns[['role', 'mean_return', 'median_return', 'success_rate', 'mean_adj_return', 'count']])

# Plot role-based returns
plt.figure(figsize=(14, 6))
plt.bar(role_returns['role'], role_returns['mean_adj_return'], color='purple')
plt.axhline(y=0, color='red', linestyle='--', label='S&P 500 Return')
plt.title('Market-Adjusted Returns by Insider Role')
plt.xlabel('Role')
plt.ylabel('Average Return - S&P 500 Return (%)')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.legend()

# Add transaction counts as text on each bar
for i, v in enumerate(role_returns['count']):
    plt.text(i, role_returns['mean_adj_return'].iloc[i] + 0.5,
             f'n={v}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('role_returns.png', dpi=300)
plt.show()

#------------------------#
# 5. Time-Based Analysis
#------------------------#
# Calculate average returns by year
yearly_returns = df_filtered.groupby('Transaction_Year').agg({
    'return_6month_pct': ['mean', 'median', 'std', 'count'],
    'spy_return_6month_pct': 'mean',
    'market_adjusted_return': 'mean'
}).reset_index()

# Flatten multi-level columns
yearly_returns.columns = ['year', 'mean_return', 'median_return', 'std_return', 'count',
                          'mean_spy_return', 'mean_adj_return']

print("\n=== YEARLY PERFORMANCE ===")
print(yearly_returns[['year', 'mean_return', 'mean_spy_return', 'mean_adj_return', 'count']])

# Plot returns over time
plt.figure(figsize=(14, 8))

plt.plot(yearly_returns['year'], yearly_returns['mean_return'], 'o-', color='blue',
         linewidth=2, markersize=8, label='Insider Returns')
plt.plot(yearly_returns['year'], yearly_returns['mean_spy_return'], 's-', color='green',
         linewidth=2, markersize=8, label='S&P 500 Returns')
plt.plot(yearly_returns['year'], yearly_returns['mean_adj_return'], '^-', color='red',
         linewidth=2, markersize=8, label='Market-Adjusted Returns')

plt.axhline(y=0, color='black', linestyle='--')
plt.title('6-Month Returns Following Insider Purchases By Year')
plt.xlabel('Year')
plt.ylabel('Average Return (%)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(yearly_returns['year'])

# Add transaction counts as text
for i, year in enumerate(yearly_returns['year']):
    plt.text(year, yearly_returns['mean_return'].iloc[i] + 2,
             f'n={yearly_returns["count"].iloc[i]}', ha='center', fontsize=8)

plt.tight_layout()
plt.savefig('yearly_returns.png', dpi=300)
plt.show()

#------------------------#
# 6. Consumer Defensive Deep Dive
#------------------------#
# Focus on Consumer Defensive Sector after filtering
consumer_defensive = df_filtered[df_filtered['sector'] == 'Consumer Defensive'].copy()

# Calculate returns by industry within Consumer Defensive
cd_industry_returns = consumer_defensive.groupby('industry').agg({
    'return_6month_pct': ['mean', 'median', 'std', 'count'],
    'market_adjusted_return': ['mean', 'median']
}).reset_index()

# Flatten multi-level columns
cd_industry_returns.columns = ['industry', 'mean_return', 'median_return', 'std_return', 'count',
                              'mean_adj_return', 'median_adj_return']

# Filter for minimum sample size
cd_industry_returns = cd_industry_returns[cd_industry_returns['count'] >= 10]
cd_industry_returns = cd_industry_returns.sort_values('mean_return', ascending=False)

print("\n=== CONSUMER DEFENSIVE INDUSTRY ANALYSIS ===")
print(cd_industry_returns[['industry', 'mean_return', 'median_return', 'mean_adj_return', 'count']])

# Plot industry returns within Consumer Defensive
plt.figure(figsize=(14, 6))
plt.bar(cd_industry_returns['industry'], cd_industry_returns['mean_return'], color='navy')
plt.axhline(y=df_filtered['return_6month_pct'].mean(), color='green', linestyle='-',
            label=f'Overall Stock Return Mean: {df_filtered["return_6month_pct"].mean():.2f}%')
plt.axhline(y=consumer_defensive['return_6month_pct'].mean(), color='orange', linestyle='-',
            label=f'Sector Mean: {consumer_defensive["return_6month_pct"].mean():.2f}%')
plt.title('Average 6-Month Returns by Industry in Consumer Defensive Sector')
plt.xlabel('Industry')
plt.ylabel('Average Return (%)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', alpha=0.3)

# Add transaction counts as text
for i, v in enumerate(cd_industry_returns['count']):
    plt.text(i, cd_industry_returns['mean_return'].iloc[i] + 0.5,
             f'n={v}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('consumer_defensive_industry.png', dpi=300)
plt.show()

#------------------------#
# 7. Success Rate Analysis
#------------------------#
# Calculate success rates and risk-adjusted returns
print("\n=== SUCCESS RATES AND RISK-ADJUSTED RETURNS ===")

# Add Sharpe ratio (risk-adjusted return)
sector_returns['sharpe_ratio'] = sector_returns['mean_adj_return'] / sector_returns['std_adj_return']

# Print sectors by success rate
success_by_sector = sector_returns.sort_values('success_rate', ascending=False)
print("\nSectors by Success Rate (% positive returns):")
print(success_by_sector[['sector', 'success_rate', 'mean_adj_return', 'sharpe_ratio', 'count']])

# Create a scatter plot of return vs. success rate
plt.figure(figsize=(12, 8))
scatter = plt.scatter(sector_returns['success_rate'], sector_returns['mean_adj_return'],
                      s=sector_returns['count']/10, # Size by sample count
                      c=sector_returns['sharpe_ratio'], # Color by Sharpe ratio
                      cmap='viridis', alpha=0.7)

plt.axhline(y=0, color='red', linestyle='--')
plt.axvline(x=20, color='red', linestyle='--')
plt.colorbar(scatter, label='Sharpe Ratio')

# Add sector labels
for i, row in sector_returns.iterrows():
    plt.annotate(row['sector'],
                 (row['success_rate'], row['mean_adj_return']),
                 xytext=(5, 5), textcoords='offset points')

plt.title('Return vs. Success Rate by Sector')
plt.xlabel('Success Rate (% Positive Returns)')
plt.ylabel('Market-Adjusted Return (%)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('success_rate_vs_return.png', dpi=300)
plt.show()

#------------------------#
# 8. Summary Statistics
#------------------------#
print("\n=== SUMMARY STATISTICS ===")

# Overall statistics
positive_pct = (df_filtered['return_6month_pct'] > 0).mean() * 100
beat_market_pct = (df_filtered['market_adjusted_return'] > 0).mean() * 100

print(f"Total insider transactions (after filtering): {df_filtered.shape[0]}")
print(f"Unique companies: {df_filtered['Ticker'].nunique()}")
print(f"Date range: {df_filtered['Transaction_Year'].min()} to {df_filtered['Transaction_Year'].max()}")
print(f"Average 6-month return: {df_filtered['return_6month_pct'].mean():.2f}%")
print(f"Median 6-month return: {df_filtered['return_6month_pct'].median():.2f}%")
print(f"Average market-adjusted return: {df_filtered['market_adjusted_return'].mean():.2f}%")
print(f"Percentage of positive returns: {positive_pct:.1f}%")
print(f"Percentage of transactions beating the S&P 500: {beat_market_pct:.1f}%")